In [27]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [28]:
def shuffle_dict(col,seed):
    key = col.unique()
    shuffled = np.array(key,copy=True)
    shuffled = shuffle(shuffled,random_state=seed)
    adict = dict(itertools.izip(key,shuffled))
    return col.apply(lambda x: adict[x])

In [29]:
def label(x):
    if x > 0.5:
        return 'H'
    elif x > 0.3:
        return 'M'
    return 'L'

In [30]:
def safe_div(x):
    if x[1] == 0:
        return 0
    return x[0] / x[1]

In [31]:
o_185 = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/o_185_trainData.csv')
o_196 = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/o_196_trainData.csv')
o_38 = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/o_38_trainData.csv')
o_313 = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/o_313_trainData.csv')
o_4550 = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/o_4550_trainData.csv')

In [32]:
countries = pd.read_csv("https://raw.githubusercontent.com/rpedsel/files/master/feature_vectors.csv",header=None)

mmg = pd.read_csv("https://raw.githubusercontent.com/rpedsel/files/master/mammographic_masses.data", names=['BIRADS', 'Age', 'Shape', 'Margin', 'Density', 'Severity'])
mmg = mmg.replace('55',5)
mmg = mmg.replace('?',np.nan).dropna().astype(int)

pokerhand = pd.read_csv("https://raw.githubusercontent.com/rpedsel/files/master/poker-hand-training-true.data", names=['S1', 'C1', 'S2', 'C2', 'S3', 'C3', 'S4', 'C4', 'S5', 'C5', 'class'])

mmold = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/mm-old-processed.csv')

ss = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/ss-regression.csv')
ss.loc[(ss['col16'] > 0) & (ss['col16'] < 1000), 'col16'] = 1
ss.loc[(ss['col16'] >= 1000) & (ss['col16'] < 1200), 'col16'] = 2
ss.loc[(ss['col16'] == 0) | (ss['col16'] >= 1200), 'col16'] = 0

game = pd.read_csv("https://raw.githubusercontent.com/rpedsel/files/master/SkillCraft1_Dataset.csv")
game = game.replace('?',np.nan).dropna()
game['Age'] = game['Age'].astype(int)
game['HoursPerWeek'] = game['HoursPerWeek'].astype(int)
game['TotalHours'] = game['TotalHours'].astype(int)
game.loc[game['ComplexAbilitiesUsed'] == 0, 'ComplexAbilitiesUsed'] = 0
game.loc[(game['ComplexAbilitiesUsed'] >= 0.00015) , 'ComplexAbilitiesUsed'] = 2
game.loc[(game['ComplexAbilitiesUsed'] > 0) & (game['ComplexAbilitiesUsed'] < 0.00015), 'ComplexAbilitiesUsed'] = 1
game['ComplexAbilitiesUsed'] = game['ComplexAbilitiesUsed'].astype(int)

earn = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/earn.csv')
flag = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/flag.csv')
cars = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/cars.csv')
abalone = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/abalone.csv')
wine_r = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/winequality-red.csv')
wine_w = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/winequality-white.csv')
credit = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/creditcard.csv')
cmc = pd.read_csv('https://raw.githubusercontent.com/rpedsel/files/master/cmc.csv')

In [34]:
def runCorrDrop_(data, cat=[], unknown=False):
    nlist = []
    table = pd.DataFrame(columns=['col_name','level','level_ratio','nunique','nunique_ratio','drop','dtype','95%in10','type'])
    for name in data:
        col = data[name]
        if col.dtype.kind in np.typecodes['AllInteger']+'uf' and col.nunique()>2:
            nlist.append(name)
        
    ndata = data[nlist]
    origCorr = ndata.corr(method='spearman').fillna(0)
    ncol = len(ndata.columns)     
    
    for name in data:
        dtype = 'numerical' if name not in cat else 'categorical'
        col = data[name]
        
        if col.count() == 0:
            table = table.append({'col_name': name, 'level': np.nan, 'level_ratio': np.nan,
                                 'nunique':col.nunique(), 'nunique_ratio':np.nan,
                                 'drop':np.nan, 'dtype':np.nan, '95%in10':np.nan, 'type':'empty'}, ignore_index=True) 
        elif name in nlist:
            in10 = col.value_counts().head(10).sum() / float(col.count()) > .95
            if len(nlist) != 1:
                orig = abs(origCorr[name])
                temp = [1]*ncol
                level = list(orig.map(label))
                lvl = (max(0,level.count('H')-1),level.count('M'),level.count('L'))
                lvl_ratio = tuple(map(lambda x: round(float(x)/(len(level)-1),4),lvl))
         
                for i in range(5):
                    ndatacopy = ndata.copy()
                    ndatacopy[name] = shuffle_dict(ndatacopy[name],i)
                    corr = ndatacopy.corr(method='spearman')[name].fillna(0)               
                    shuf = abs(corr)
                    temp = [min(x) for x in zip(shuf, temp)]
                #pair = zip((orig-temp),orig)
                #drop = map(safe_div, pair)
                #p = orig*drop

                p = orig-temp
                table = table.append({'col_name':name, 'level':lvl, 'level_ratio':lvl_ratio,
                                     'nunique':col.nunique(), 'nunique_ratio':round(float(col.nunique()),5)/col.count(),
                                     'drop':(round(np.mean(p),4),round(np.median(p),4),round(np.max(p),4),round(np.min(p),4)),
                                      'dtype':col.dtype,'95%in10':in10, 'type':dtype}, ignore_index=True)
            else:
                table = table.append({'col_name':name, 'level':np.nan, 'level_ratio':np.nan,
                                     'nunique':col.nunique(), 'nunique_ratio':round(float(col.nunique()),5)/col.count(),
                                     'drop':np.nan, 'dtype':col.dtype,'95%in10':in10, 'type':dtype}, ignore_index=True)
        else:
            in10 = col.value_counts().head(10).sum() / float(col.count()) > .95
            if col.nunique() == 2:
                table = table.append({'col_name':name, 'level': np.nan, 'level_ratio': np.nan,
                                    'nunique': col.nunique(), 'nunique_ratio':np.nan, 'drop':np.nan, 
                                    'dtype':col.dtype,'95%in10':in10, 'type':'binary column'}, ignore_index=True)
            else:
                table = table.append({'col_name': name, 'level': np.nan, 'level_ratio': np.nan,
                                 'nunique':col.nunique(), 'nunique_ratio':round(float(col.nunique()),5)/col.count(),
                                 'drop':np.nan, 'dtype':col.dtype,'95%in10':in10, 'type':'string column'}, ignore_index=True)        
    if unknown:
        table['type']='unknown'
    return table

In [35]:
runCorrDrop_(earn,['workclass','education','education-num','marital-status','occupation','relationship','race','sex','native-country','earn']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
9,sex,NaN,NaN,2,NaN,NaN,object,True,binary column
14,earn,NaN,NaN,2,NaN,NaN,object,True,binary column
4,education-num,"(0, 0, 5)","(0.0, 0.0, 1.0)",16,0.000491,"(0.0695, 0.0555, 0.1596, 0.0)",int64,False,categorical
0,age,"(0, 0, 5)","(0.0, 0.0, 1.0)",73,0.002242,"(0.0686, 0.0665, 0.1123, 0.0)",int64,False,numerical
2,fnlwgt,"(0, 0, 5)","(0.0, 0.0, 1.0)",21648,0.664844,"(0.0236, 0.0132, 0.0761, 0.0)",int64,False,numerical
10,capital-gain,"(0, 0, 5)","(0.0, 0.0, 1.0)",119,0.003655,"(0.0637, 0.0725, 0.118, 0.0)",int64,True,numerical
11,capital-loss,"(0, 0, 5)","(0.0, 0.0, 1.0)",92,0.002825,"(0.0407, 0.0534, 0.0677, 0.0)",int64,True,numerical
12,hours-per-week,"(0, 0, 5)","(0.0, 0.0, 1.0)",94,0.002887,"(0.0746, 0.0756, 0.1532, 0.0)",int64,False,numerical
1,workclass,NaN,NaN,9,0.000276,NaN,object,True,string column
3,education,NaN,NaN,16,0.000491,NaN,object,False,string column


In [36]:
runCorrDrop_(pokerhand,['S1','S2','S3','S4','S5','class']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,S1,"(0, 0, 10)","(0.0, 0.0, 1.0)",4,0.00016,"(0.0057, 0.0065, 0.0103, 0.0)",int64,True,categorical
2,S2,"(0, 0, 10)","(0.0, 0.0, 1.0)",4,0.00016,"(0.0059, 0.0047, 0.0161, 0.0)",int64,True,categorical
4,S3,"(0, 0, 10)","(0.0, 0.0, 1.0)",4,0.00016,"(0.0054, 0.0038, 0.0145, -0.0018)",int64,True,categorical
6,S4,"(0, 0, 10)","(0.0, 0.0, 1.0)",4,0.00016,"(0.0057, 0.0044, 0.0162, 0.0)",int64,True,categorical
8,S5,"(0, 0, 10)","(0.0, 0.0, 1.0)",4,0.00016,"(0.0062, 0.0043, 0.0228, -0.0022)",int64,True,categorical
10,class,"(0, 0, 10)","(0.0, 0.0, 1.0)",10,0.00040,"(0.0021, 0.0009, 0.0068, -0.0007)",int64,True,categorical
1,C1,"(0, 0, 10)","(0.0, 0.0, 1.0)",13,0.00052,"(0.0076, 0.0045, 0.0219, 0.0)",int64,False,numerical
3,C2,"(0, 0, 10)","(0.0, 0.0, 1.0)",13,0.00052,"(0.0076, 0.0077, 0.0229, -0.0031)",int64,False,numerical
5,C3,"(0, 0, 10)","(0.0, 0.0, 1.0)",13,0.00052,"(0.0106, 0.0091, 0.026, 0.0)",int64,False,numerical
7,C4,"(0, 0, 10)","(0.0, 0.0, 1.0)",13,0.00052,"(0.0079, 0.0083, 0.0146, 0.0)",int64,False,numerical


In [37]:
runCorrDrop_(mmg,['BIRADS','Shape','Margin','Density','Severity']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
5,Severity,NaN,NaN,2,NaN,NaN,int64,True,binary column
0,BIRADS,"(2, 1, 1)","(0.5, 0.25, 0.25)",6,0.007229,"(0.033, 0.0433, 0.0801, -0.004)",int64,True,categorical
2,Shape,"(2, 1, 1)","(0.5, 0.25, 0.25)",4,0.004819,"(0.057, 0.0468, 0.1226, 0.0)",int64,True,categorical
3,Margin,"(2, 1, 1)","(0.5, 0.25, 0.25)",5,0.006024,"(0.3282, 0.3611, 0.7042, 0.0)",int64,True,categorical
4,Density,"(0, 0, 4)","(0.0, 0.0, 1.0)",4,0.004819,"(0.0423, 0.0385, 0.0953, 0.0)",int64,True,categorical
1,Age,"(0, 3, 1)","(0.0, 0.75, 0.25)",72,0.086747,"(0.2154, 0.3256, 0.365, 0.0)",int64,False,numerical


In [38]:
runCorrDrop_(game,['GameID','LeagueIndex']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,GameID,"(0, 0, 19)","(0.0, 0.0, 1.0)",3338,1.000000,"(0.0384, 0.0392, 0.0959, 0.0)",int64,False,categorical
1,LeagueIndex,"(5, 5, 9)","(0.2632, 0.2632, 0.4737)",7,0.002097,"(0.2967, 0.2487, 0.624, 0.0)",int64,True,categorical
2,Age,"(0, 0, 19)","(0.0, 0.0, 1.0)",28,0.008388,"(0.0715, 0.0686, 0.1796, 0.0)",int64,False,numerical
3,HoursPerWeek,"(0, 1, 18)","(0.0, 0.0526, 0.9474)",32,0.009587,"(0.1154, 0.0885, 0.4408, 0.0)",int64,False,numerical
4,TotalHours,"(0, 5, 14)","(0.0, 0.2632, 0.7368)",237,0.071001,"(0.201, 0.1596, 0.4967, 0.0)",int64,False,numerical
8,UniqueHotkeys,"(0, 6, 13)","(0.0, 0.3158, 0.6842)",11,0.003295,"(0.1882, 0.1661, 0.3894, 0.0)",int64,True,numerical
15,TotalMapExplored,"(1, 3, 15)","(0.0526, 0.1579, 0.7895)",52,0.015578,"(0.2083, 0.1961, 0.576, -0.0033)",int64,False,numerical
17,UniqueUnitsMade,"(2, 2, 15)","(0.1053, 0.1053, 0.7895)",12,0.003595,"(0.1655, 0.125, 0.5538, 0.0)",int64,True,numerical
19,ComplexAbilitiesUsed,"(1, 1, 17)","(0.0526, 0.0526, 0.8947)",3,0.000899,"(0.0879, 0.0617, 0.3057, 0.0)",int64,True,numerical
5,APM,"(6, 5, 8)","(0.3158, 0.2632, 0.4211)",3317,0.993709,"(0.3829, 0.3371, 0.8112, 0.0)",float64,False,numerical


In [39]:
runCorrDrop_(o_185.fillna(0),['d3mIndex','Player','Position']).sort_values(['type','dtype'])
# Player -> string
# Position -> string

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,d3mIndex,"(0, 0, 15)","(0.0, 0.0, 1.0)",1073,1.000000,"(0.0005, 0.0008, 0.0182, -0.0201)",int64,False,categorical
2,Number_seasons,"(7, 3, 5)","(0.4667, 0.2, 0.3333)",17,0.015843,"(0.42, 0.4224, 0.7571, 0.0)",int64,False,numerical
3,Games_played,"(10, 3, 2)","(0.6667, 0.2, 0.1333)",836,0.779124,"(0.5812, 0.6167, 0.9603, -0.0019)",int64,False,numerical
4,At_bats,"(10, 3, 2)","(0.6667, 0.2, 0.1333)",1011,0.942218,"(0.5887, 0.622, 0.9809, -0.0093)",int64,False,numerical
5,Runs,"(12, 1, 2)","(0.8, 0.0667, 0.1333)",716,0.667288,"(0.5958, 0.5989, 0.9492, -0.007)",int64,False,numerical
6,Hits,"(12, 1, 2)","(0.8, 0.0667, 0.1333)",856,0.797763,"(0.604, 0.6508, 0.9774, 0.0)",int64,False,numerical
7,Doubles,"(12, 1, 2)","(0.8, 0.0667, 0.1333)",391,0.364399,"(0.6061, 0.6559, 0.9503, -0.0068)",int64,False,numerical
8,Triples,"(8, 3, 4)","(0.5333, 0.2, 0.2667)",171,0.159366,"(0.4507, 0.4947, 0.787, 0.0)",int64,False,numerical
9,Home_runs,"(9, 4, 2)","(0.6, 0.2667, 0.1333)",277,0.258155,"(0.4631, 0.5271, 0.8222, 0.0)",int64,False,numerical
10,RBIs,"(13, 0, 2)","(0.8667, 0.0, 0.1333)",695,0.647717,"(0.601, 0.6793, 0.9131, 0.0)",int64,False,numerical


In [40]:
runCorrDrop_(o_196.dropna(),['d3mIndex','cylinders','model','origin']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,d3mIndex,"(0, 0, 7)","(0.0, 0.0, 1.0)",293,1.000000,"(0.0479, 0.0496, 0.0927, 0.0)",int64,False,categorical
1,cylinders,"(4, 2, 1)","(0.5714, 0.2857, 0.1429)",5,0.017065,"(0.3255, 0.3658, 0.6077, 0.0)",int64,True,categorical
6,model,"(0, 3, 4)","(0.0, 0.4286, 0.5714)",13,0.044369,"(0.2162, 0.2524, 0.3844, 0.0)",int64,False,categorical
7,origin,"(4, 0, 3)","(0.5714, 0.0, 0.4286)",3,0.010239,"(0.3097, 0.3386, 0.6136, 0.0)",int64,True,categorical
2,displacement,"(4, 2, 1)","(0.5714, 0.2857, 0.1429)",75,0.255973,"(0.5026, 0.5053, 0.9357, 0.0)",float64,False,numerical
3,horsepower,"(5, 1, 1)","(0.7143, 0.1429, 0.1429)",86,0.293515,"(0.507, 0.5573, 0.8658, 0.0)",float64,False,numerical
4,weight,"(4, 1, 2)","(0.5714, 0.1429, 0.2857)",268,0.914676,"(0.4844, 0.4532, 0.9366, 0.0)",float64,False,numerical
5,acceleration,"(1, 3, 3)","(0.1429, 0.4286, 0.4286)",89,0.303754,"(0.2824, 0.274, 0.6118, 0.0)",float64,False,numerical


In [41]:
cat = [
'd3mIndex',
'sex',

'on_thyroxine',
'query_on_thyroxine', 
'on_antithyroid_medication',
'sick',
'pregnant',    
'thyroid_surgery', 
'I131_treatment',  
'query_hypothyroid',   
'query_hyperthyroid',
'lithium', 
'goitre',  
'tumor',   
'hypopituitary',
'psych',
'TSH_measured',
'T3_measured',
'TT4_measured',
'T4U_measured',
'FTI_measured',
'TBG_measured',

'referral_source',
'label'
]
runCorrDrop_(o_38.dropna(axis=1),cat).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
1,on_thyroxine,NaN,NaN,2,NaN,NaN,object,True,binary column
2,query_on_thyroxine,NaN,NaN,2,NaN,NaN,object,True,binary column
3,on_antithyroid_medication,NaN,NaN,2,NaN,NaN,object,True,binary column
4,sick,NaN,NaN,2,NaN,NaN,object,True,binary column
5,pregnant,NaN,NaN,2,NaN,NaN,object,True,binary column
6,thyroid_surgery,NaN,NaN,2,NaN,NaN,object,True,binary column
7,I131_treatment,NaN,NaN,2,NaN,NaN,object,True,binary column
8,query_hypothyroid,NaN,NaN,2,NaN,NaN,object,True,binary column
9,query_hyperthyroid,NaN,NaN,2,NaN,NaN,object,True,binary column
10,lithium,NaN,NaN,2,NaN,NaN,object,True,binary column


In [42]:
runCorrDrop_(o_313.dropna(),['d3mIndex', 'LRS-name','ID-type','label']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,d3mIndex,"(0, 0, 101)","(0.0, 0.0, 1.0)",428,1.000000,"(0.0368, 0.0379, 0.0965, -0.0161)",int64,False,categorical
2,ID-type,"(0, 52, 49)","(0.0, 0.5149, 0.4851)",4,0.009346,"(0.1851, 0.2059, 0.343, -0.0113)",int64,True,categorical
3,Right-Ascension,"(1, 0, 100)","(0.0099, 0.0, 0.9901)",421,0.983645,"(0.0393, 0.0247, 0.5831, -0.0236)",float64,False,numerical
4,Declination,"(1, 1, 99)","(0.0099, 0.0099, 0.9802)",427,0.997664,"(0.0679, 0.0349, 0.6142, -0.015)",float64,False,numerical
5,Scale_Factor,"(4, 44, 53)","(0.0396, 0.4356, 0.5248)",339,0.792056,"(0.2524, 0.2635, 0.9656, -0.0209)",float64,False,numerical
6,Blue_base_1,"(4, 45, 52)","(0.0396, 0.4455, 0.5149)",382,0.892523,"(0.2602, 0.2554, 0.9811, -0.0324)",float64,False,numerical
7,Blue_base_2,"(4, 43, 54)","(0.0396, 0.4257, 0.5347)",389,0.908879,"(0.2572, 0.2569, 0.9846, -0.0183)",float64,False,numerical
8,Red_base_1,"(4, 39, 58)","(0.0396, 0.3861, 0.5743)",363,0.848131,"(0.2418, 0.2472, 0.9773, -0.0147)",float64,False,numerical
9,Red_base_2,"(4, 47, 50)","(0.0396, 0.4653, 0.495)",356,0.831776,"(0.2639, 0.2807, 0.9815, -0.0101)",float64,False,numerical
10,blue-band-flux_1,"(54, 16, 31)","(0.5347, 0.1584, 0.3069)",428,1.000000,"(0.4414, 0.5012, 0.9734, -0.017)",float64,False,numerical


In [43]:
runCorrDrop_(o_4550.dropna(),['d3mIndex','MouseID','Genotype', 'Treatment','Behavior','label']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
79,Genotype,NaN,NaN,2,NaN,NaN,object,True,binary column
80,Treatment,NaN,NaN,2,NaN,NaN,object,True,binary column
81,Behavior,NaN,NaN,2,NaN,NaN,object,True,binary column
0,d3mIndex,"(0, 0, 77)","(0.0, 0.0, 1.0)",444,1.000000,"(0.0288, 0.0275, 0.0971, -0.0139)",int64,False,categorical
2,DYRK1A_N,"(13, 19, 45)","(0.1688, 0.2468, 0.5844)",444,1.000000,"(0.2645, 0.222, 0.9275, -0.0213)",float64,False,numerical
3,ITSN1_N,"(17, 21, 39)","(0.2208, 0.2727, 0.5065)",444,1.000000,"(0.2973, 0.2952, 0.9288, -0.0092)",float64,False,numerical
4,BDNF_N,"(32, 18, 27)","(0.4156, 0.2338, 0.3506)",444,1.000000,"(0.4094, 0.4152, 0.8373, 0.0)",float64,False,numerical
5,NR1_N,"(26, 25, 26)","(0.3377, 0.3247, 0.3377)",444,1.000000,"(0.4027, 0.3904, 0.9165, 0.0)",float64,False,numerical
6,NR2A_N,"(23, 26, 28)","(0.2987, 0.3377, 0.3636)",444,1.000000,"(0.3808, 0.3647, 0.894, -0.0012)",float64,False,numerical
7,pAKT_N,"(19, 23, 35)","(0.2468, 0.2987, 0.4545)",444,1.000000,"(0.3387, 0.3248, 0.8777, 0.0)",float64,False,numerical


In [44]:
runCorrDrop_(flag,['name','landmass','zone','language','religion','red','green','blue','gold','white','black','orange','mainhue','crecent','triangle','icon','animate','text','topleft','botright']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
10,red,NaN,NaN,2,NaN,NaN,int64,True,binary column
11,green,NaN,NaN,2,NaN,NaN,int64,True,binary column
12,blue,NaN,NaN,2,NaN,NaN,int64,True,binary column
13,gold,NaN,NaN,2,NaN,NaN,int64,True,binary column
14,white,NaN,NaN,2,NaN,NaN,int64,True,binary column
15,black,NaN,NaN,2,NaN,NaN,int64,True,binary column
16,orange,NaN,NaN,2,NaN,NaN,int64,True,binary column
20,saltires,NaN,NaN,2,NaN,NaN,int64,True,binary column
23,crecent,NaN,NaN,2,NaN,NaN,int64,True,binary column
24,triangle,NaN,NaN,2,NaN,NaN,int64,True,binary column


In [45]:
runCorrDrop_(cars,['buying','maint','doors','persons','lug_boot','safety','class']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,buying,"(0, 0, 6)","(0.0, 0.0, 1.0)",4,0.002315,"(0.0169, 0.0, 0.1181, 0.0)",int64,True,categorical
1,maint,"(0, 0, 6)","(0.0, 0.0, 1.0)",4,0.002315,"(0.0104, 0.0, 0.0725, 0.0)",int64,True,categorical
2,doors,"(0, 0, 6)","(0.0, 0.0, 1.0)",4,0.002315,"(0.0042, 0.0, 0.0295, 0.0)",int64,True,categorical
3,persons,"(0, 1, 5)","(0.0, 0.1667, 0.8333)",3,0.001736,"(0.0538, 0.0, 0.3767, 0.0)",int64,True,categorical
4,lug_boot,"(0, 0, 6)","(0.0, 0.0, 1.0)",3,0.001736,"(0.0137, 0.0, 0.0959, 0.0)",int64,True,categorical
5,safety,"(0, 1, 5)","(0.0, 0.1667, 0.8333)",3,0.001736,"(0.0463, 0.0, 0.3243, 0.0)",int64,True,categorical
6,class,"(0, 2, 4)","(0.0, 0.3333, 0.6667)",4,0.002315,"(0.1494, 0.1769, 0.2948, 0.0)",int64,True,categorical


In [46]:
runCorrDrop_(abalone,['sex','rings']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,sex,"(0, 8, 0)","(0.0, 1.0, 0.0)",3,0.000718,"(0.3884, 0.4322, 0.467, 0.0)",int64,True,categorical
8,rings,"(7, 1, 0)","(0.875, 0.125, 0.0)",28,0.006703,"(0.4467, 0.5099, 0.5734, 0.0)",int64,False,categorical
1,length,"(7, 1, 0)","(0.875, 0.125, 0.0)",134,0.032080,"(0.7475, 0.9464, 0.9825, 0.0)",float64,False,numerical
2,diameter,"(7, 1, 0)","(0.875, 0.125, 0.0)",111,0.026574,"(0.7472, 0.9431, 0.9788, 0.0)",float64,False,numerical
3,height,"(7, 1, 0)","(0.875, 0.125, 0.0)",51,0.012210,"(0.717, 0.8816, 0.9136, 0.0)",float64,False,numerical
4,whole,"(7, 1, 0)","(0.875, 0.125, 0.0)",2429,0.581518,"(0.7592, 0.9579, 0.9686, 0.0)",float64,False,numerical
5,shucked,"(7, 1, 0)","(0.875, 0.125, 0.0)",1515,0.362701,"(0.73, 0.9111, 0.9698, 0.0)",float64,False,numerical
6,viscera,"(7, 1, 0)","(0.875, 0.125, 0.0)",880,0.210678,"(0.7441, 0.9312, 0.9752, 0.0)",float64,False,numerical
7,shell,"(7, 1, 0)","(0.875, 0.125, 0.0)",926,0.221690,"(0.7467, 0.9107, 0.9563, 0.0)",float64,False,numerical


In [47]:
runCorrDrop_(credit,['A1','A4','A5','A6','A8','A9','A11','A12','A15']).sort_values('type').sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,A1,NaN,NaN,2,NaN,NaN,int64,True,binary column
7,A8,NaN,NaN,2,NaN,NaN,int64,True,binary column
8,A9,NaN,NaN,2,NaN,NaN,int64,True,binary column
10,A11,NaN,NaN,2,NaN,NaN,int64,True,binary column
14,A15,NaN,NaN,2,NaN,NaN,int64,True,binary column
3,A4,"(0, 0, 9)","(0.0, 0.0, 1.0)",3,0.004348,"(0.0009, 0.0, 0.0052, 0.0)",int64,True,categorical
4,A5,"(0, 1, 8)","(0.0, 0.1111, 0.8889)",14,0.020290,"(0.11, 0.0519, 0.3225, 0.0)",int64,False,categorical
5,A6,"(0, 2, 7)","(0.0, 0.2222, 0.7778)",8,0.011594,"(0.0947, 0.0476, 0.3246, 0.0)",int64,True,categorical
11,A12,"(0, 0, 9)","(0.0, 0.0, 1.0)",3,0.004348,"(0.0006, 0.0, 0.0064, 0.0)",int64,True,categorical
9,A10,"(0, 2, 7)","(0.0, 0.2222, 0.7778)",23,0.033333,"(0.1783, 0.1689, 0.4186, 0.0)",int64,False,numerical


In [48]:
runCorrDrop_(cmc,['wife_edu','husband_edu','religion','isworking','occupation','std_living','media','class']).sort_values(['type','dtype'])

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
4,religion,NaN,NaN,2,NaN,NaN,int64,True,binary column
5,isworking,NaN,NaN,2,NaN,NaN,int64,True,binary column
8,media,NaN,NaN,2,NaN,NaN,int64,True,binary column
1,wife_edu,"(1, 2, 3)","(0.1667, 0.3333, 0.5)",4,0.002716,"(0.1982, 0.1112, 0.5506, -0.0504)",int64,True,categorical
2,husband_edu,"(1, 2, 3)","(0.1667, 0.3333, 0.5)",4,0.002716,"(0.1866, 0.1416, 0.5323, 0.0)",int64,True,categorical
6,occupation,"(0, 2, 4)","(0.0, 0.3333, 0.6667)",4,0.002716,"(0.1241, 0.1035, 0.292, -0.0059)",int64,True,categorical
7,std_living,"(0, 2, 4)","(0.0, 0.3333, 0.6667)",4,0.002716,"(0.1116, 0.1466, 0.2105, 0.0)",int64,True,categorical
0,age,"(1, 0, 5)","(0.1667, 0.0, 0.8333)",34,0.023082,"(0.1534, 0.1391, 0.5475, 0.0)",int64,False,numerical
3,num_child,"(1, 0, 5)","(0.1667, 0.0, 0.8333)",15,0.010183,"(0.1367, 0.1103, 0.5195, 0.0)",int64,True,numerical
9,method,"(0, 0, 6)","(0.0, 0.0, 1.0)",3,0.002037,"(0.0397, 0.0, 0.1351, 0.0)",int64,True,numerical


In [49]:
runCorrDrop_(ss,unknown=True)

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,col1,"(2, 4, 3)","(0.2222, 0.4444, 0.3333)",100,0.015699,"(0.3111, 0.3682, 0.5409, 0.0)",int64,False,unknown
1,col2,NaN,NaN,2,NaN,NaN,int64,True,unknown
2,col3,NaN,NaN,2,NaN,NaN,int64,True,unknown
3,col4,NaN,NaN,2,NaN,NaN,int64,True,unknown
4,col5,NaN,NaN,2,NaN,NaN,int64,True,unknown
5,col6,NaN,NaN,2,NaN,NaN,int64,True,unknown
6,col7,"(0, 2, 7)","(0.0, 0.2222, 0.7778)",4,0.000628,"(-0.0057, -0.0063, 0.0005, -0.011)",int64,True,unknown
7,col8,NaN,NaN,2,NaN,NaN,int64,True,unknown
8,col9,"(4, 2, 3)","(0.4444, 0.2222, 0.3333)",48,0.007535,"(0.3513, 0.3281, 0.6228, 0.0)",int64,False,unknown
9,col10,"(2, 3, 4)","(0.2222, 0.3333, 0.4444)",41,0.006436,"(0.2993, 0.2492, 0.5674, 0.0)",int64,False,unknown


In [50]:
runCorrDrop_(mmold,unknown=True)

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,1,"(17, 1, 3)","(0.8095, 0.0476, 0.1429)",78,0.300000,"(0.613, 0.7129, 0.8925, 0.0)",int64,False,unknown
1,2,"(14, 5, 2)","(0.6667, 0.2381, 0.0952)",48,0.184615,"(0.5853, 0.6441, 0.9783, 0.0)",int64,False,unknown
2,3,"(13, 6, 2)","(0.619, 0.2857, 0.0952)",52,0.200000,"(0.4325, 0.4499, 0.714, 0.0)",float64,False,unknown
3,4,"(14, 5, 2)","(0.6667, 0.2381, 0.0952)",63,0.242308,"(0.585, 0.5727, 0.9735, 0.0)",int64,False,unknown
4,5,"(12, 6, 3)","(0.5714, 0.2857, 0.1429)",26,0.100000,"(0.5438, 0.5411, 0.9863, 0.0)",float64,False,unknown
5,6,"(12, 6, 3)","(0.5714, 0.2857, 0.1429)",74,0.284615,"(0.4974, 0.5009, 0.8488, 0.0)",int64,False,unknown
6,7,"(7, 7, 7)","(0.3333, 0.3333, 0.3333)",5,0.019231,"(0.3018, 0.3478, 0.5283, -0.0052)",float64,True,unknown
7,8,"(14, 5, 2)","(0.6667, 0.2381, 0.0952)",35,0.134615,"(0.5525, 0.6011, 0.978, 0.0)",int64,False,unknown
8,9,"(14, 5, 2)","(0.6667, 0.2381, 0.0952)",48,0.184615,"(0.5574, 0.5403, 0.9717, -0.027)",int64,False,unknown
9,10,"(14, 5, 2)","(0.6667, 0.2381, 0.0952)",111,0.426923,"(0.5784, 0.7049, 0.9539, 0.0)",int64,False,unknown


In [51]:
runCorrDrop_(countries,unknown=True)

,col_name,level,level_ratio,nunique,nunique_ratio,drop,dtype,95%in10,type
0,0,"(1, 0, 3)","(0.25, 0.0, 0.75)",5,0.003307,"(0.1426, 0.1102, 0.4487, -0.0052)",int64,True,unknown
1,1,"(2, 0, 2)","(0.5, 0.0, 0.5)",10,0.006614,"(0.1429, 0.011, 0.3591, 0.0)",int64,True,unknown
2,2,NaN,NaN,1,0.000661,NaN,int64,True,unknown
3,3,"(1, 0, 3)","(0.25, 0.0, 0.75)",7,0.004630,"(0.018, 0.0005, 0.0617, 0.0)",int64,True,unknown
4,4,NaN,NaN,2,NaN,NaN,int64,True,unknown
5,5,"(0, 0, 4)","(0.0, 0.0, 1.0)",17,0.011243,"(0.0128, 0.0339, 0.0793, -0.0952)",int64,True,unknown
6,6,"(0, 0, 4)","(0.0, 0.0, 1.0)",8,0.005291,"(0.0228, 0.0124, 0.0723, 0.0)",int64,True,unknown
7,7,NaN,NaN,2,NaN,NaN,int64,True,unknown
8,8,NaN,NaN,2,NaN,NaN,int64,True,unknown
